In [7]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
frameWidth = 640
frameHeight = 480
cap = cv.VideoCapture(0)
cap.set(3,frameWidth)
cap.set(4, frameHeight)
calculatedArea = frameWidth*frameHeight

def empty(a):
    pass

def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range (0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (0,0), None, scale, scale)
                else:
                    imgArray[x][y] = cv.resize(imgArray[x][y],(imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv.cvtColor(imgArray[x][y], cv.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0,rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv.resize(imgArray[x], (0,0), None, scale, scale)
            else:
                imgArray[x] = cv.resize(imgArray[x],(imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv.cvtColor(imgArray[x], cv.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
        
    return ver

cv.namedWindow("Parameters")
cv.resizeWindow("Parameters", 640, 240)
cv.createTrackbar("threshold1", "Parameters", 100, 255, empty)
cv.createTrackbar("threshold2", "Parameters", 220, 225, empty)
# 0: top left, 1: top right, 2: bottom left, 3: bottom right
def reorder (a):
    b = np.zeros_like(a)
    a = a.reshape((4,2))
    add = a.sum(1)
    b[0] = a[np.argmin(add)]
    b[3] = a[np.argmax(add)]
    diff = np.diff(a, axis = 1)
    b[1] = a[np.argmin(diff)]
    b[2] = a[np.argmax(diff)]
    return b

def getContours(img, imgContour):
    # CHAin_APPROX_simple
    contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    imgOut = imgContour.copy()
    shape = [4]
    for cnt in contours:
        area = cv.contourArea(cnt)
        peri = cv.arcLength(cnt, True)
        approx = cv.approxPolyDP(cnt, 0.02 *peri, True)

        if(area > calculatedArea*.25 and (len(approx) in shape)):
            cv.drawContours(imgContour, cnt, -1, (255, 0,255), 1)
            x, y, w, h = cv.boundingRect(approx)
            approxReorder = reorder(approx)
            # cv.rectangle(imgContour, (x, y), (x+w, y+h), (0,255, 0), 5)
            cv.putText(imgContour, "Points: " + str(len(approx)), (x + 20, y+20), cv.FONT_HERSHEY_COMPLEX, .5, (0,255,0),2)
            cv.putText(imgContour, "Area: " + str(int(area/calculatedArea*100)) + "%", (x+20, y+45), cv.FONT_HERSHEY_COMPLEX, .7, (0,255,0),2)
            cv.putText(imgContour, "x: " + str(x) + " y: " + str(y), (x+20, y+70), cv.FONT_HERSHEY_COMPLEX, .5, (0,255,0),2)
            cv.putText(imgContour, "p1: " + str(approxReorder[0]), (x-20, y-20), cv.FONT_HERSHEY_COMPLEX, .5, (0,255,0),2)
            cv.putText(imgContour, "p2: " + str(approxReorder[1]), (x+w-80, y-20), cv.FONT_HERSHEY_COMPLEX, .5, (0,255,0),2)
            cv.putText(imgContour, "p3: " + str(approxReorder[2]), (x-20, y+h+20), cv.FONT_HERSHEY_COMPLEX, .5, (0,255,0),2)
            cv.putText(imgContour, "p4: " + str(approxReorder[3]), (x+w-80, y+h+20), cv.FONT_HERSHEY_COMPLEX, .5, (0,255,0),2)
            

            pts1 = np.float32(approxReorder)
            approxReorder = approxReorder.reshape((4,2))
            x = int(frameWidth*.8)
            y = int(x*1.4142)
            approxReorder = approxReorder.reshape((2,4))
            pts2 = np.float32([[0,0], [x,0], [0,y],[x,y]])

            M = cv.getPerspectiveTransform(pts1, pts2)

            dst = cv.warpPerspective(imgOut, M, (x,y))

            cv.imshow("output", dst)            


while True:
    success, img = cap.read()
    imgContour = img.copy()
    imgBlur = cv.GaussianBlur(img, (7,7),1)
    imgGray = cv.cvtColor(imgBlur, cv.COLOR_BGR2GRAY)
    threshold1 = cv.getTrackbarPos("threshold1", "Parameters")
    threshold2 = cv.getTrackbarPos("threshold2", "Parameters")
    imgCanny = cv.Canny(imgGray, threshold1, threshold2)

    kernel = np.ones((5,5))
    imgDil =  cv.dilate(imgCanny, kernel, iterations = 1)
    getContours(imgDil, imgContour)
    cv.putText(img, "original", (0,20), cv.FONT_HERSHEY_COMPLEX, .7, (255,0,0),2)
    cv.putText(imgBlur, "GaussianBlur (7,7)", (0,20), cv.FONT_HERSHEY_COMPLEX, .7, (255,0,0),2)
    cv.putText(imgGray, "GrayScale", (0,20), cv.FONT_HERSHEY_COMPLEX, .7, (255,0,0),2)
    cv.putText(imgCanny, "Canny " + str(threshold1) + " " +str(threshold2), (0,20), cv.FONT_HERSHEY_COMPLEX, .7, (255,0,0),2)
    cv.putText(imgDil, "Dilate (5,5)", (0,20), cv.FONT_HERSHEY_COMPLEX, .7, (255,0,0),2)
    cv.putText(imgContour, "Contour", (0,20), cv.FONT_HERSHEY_COMPLEX, .7, (255,0,0),2)
    imgStack = stackImages(0.7,([img, imgBlur, imgGray], 
                                [imgCanny, imgDil, imgContour]) )
    
    
    
    cv.imshow("result",imgStack)
    if(cv.waitKey(1) & 0xFF == ord('q')):
        cv.destroyAllWindows()
        break